In [5]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imgen = ImageDataGenerator(rescale=1/255)

train_gen = imgen.flow_from_directory('data/train', target_size=(150,150), batch_size=32)

val_gen = imgen.flow_from_directory('data/val', target_size=(150,150), batch_size=32)


Found 4606 images belonging to 12 classes.
Found 144 images belonging to 12 classes.


In [4]:
from keras.applications.vgg16 import VGG16

vgg_model = VGG16(include_top=False, input_shape=(150, 150, 3))

vgg_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

for layer in vgg_model.layers:
    layer.trainable = False

fine_tune = Sequential(layers = vgg_model.layers)

fine_tune.add(Flatten())
fine_tune.add(Dense(12, activation='softmax'))

In [7]:
fine_tune.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['acc']
)

fine_tune.fit(
    train_gen,
    steps_per_epoch=4606//32,
    epochs=8,
    validation_data=val_gen,
    validation_steps=144//32
)

c:\Users\jimin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - acc: 0.3360 - loss: 2.0103 - val_acc: 0.5156 - val_loss: 1.4567
Epoch 2/8
  1/143 ━━━━━━━━━━━━━━━━━━━━ 4:10 2s/step - acc: 0.6250 - loss: 1.2154

c:\Users\jimin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - acc: 0.6250 - loss: 1.2154 - val_acc: 0.5391 - val_loss: 1.4282
Epoch 3/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - acc: 0.6702 - loss: 1.1039 - val_acc: 0.5703 - val_loss: 1.2077
Epoch 4/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - acc: 0.7188 - loss: 0.8758 - val_acc: 0.5859 - val_loss: 1.1903
Epoch 5/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - acc: 0.7560 - loss: 0.8488 - val_acc: 0.6797 - val_loss: 0.9753
Epoch 6/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step - acc: 0.6250 - loss: 0.8879 - val_acc: 0.6875 - val_loss: 1.0099
Epoch 7/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 273s 2s/step - acc: 0.8096 - loss: 0.7036 - val_acc: 0.6719 - val_loss: 0.9769
Epoch 8/8
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - acc: 0.8125 - loss: 0.8777 - val_acc: 0.6406 - val_loss: 1.0155
